# Preprocessing transactions

In this notebook we will be preprocessing the transactions file. There are four main steps:

- Data preprocessing / cleaning

- Spliting the data in development (training) and out-of-time (test) samples

- Create the target for each sample

- Save preprocessed datasets to S3

In [14]:
!python3 -m pip install awswrangler

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 183 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 61 kB/s s eta 0:00:01
     |████████████████████████████████| 43 kB 296 kB/s  eta 0:00:01
     |████████████████████████████████| 104 kB 116.3 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 105.2 MB/s eta 0:00:01
  Attempting uninstall: asn1crypto
    Found existing installation: asn1crypto 1.3.0
    Uninstalling asn1crypto-1.3.0:
      Successfully uninstalled asn1crypto-1.3.0
  Attemptin

In [15]:
import io
import gc
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import awswrangler as wr
import boto3

def read_from_s3(bucket, filepath, file_type="csv"):
    if file_type == "csv":
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=filepath)
        csv_string = obj['Body'].read().decode('utf-8')
        return pd.read_csv(io.StringIO(csv_string))
    elif file_type == "parquet":
        buffer = io.BytesIO()
        s3 = boto3.resource('s3')
        obj = s3.Object(bucket, filepath)
        obj.download_fileobj(buffer)
        return pd.read_parquet(buffer)
    else:
        raise ValueError("File format is not supported")

In [2]:
# Read full transactions file from S3
bucket = 'capstone-transaction-data'
df = read_from_s3(bucket, 'raw_data/transactions.parquet', 'parquet')
print(df.shape)

(317376998, 11)


## Data preprocessing

Following from the EDA in the last notebook, we will be preprocessing the full dataset:

- Cutoff in march 2013

- Removing returns (transactions with negative amount)

- Remove some columns that wont be used

In [3]:
cols = [
    'id', 'chain', 'dept', 'category', 'company', 'brand',
    'date', 'purchasequantity', 'purchaseamount'
]
mask = (df.date <= 20130331) & (df.purchaseamount > 0)
df = df.loc[mask, cols]
print(df.shape)

(273618974, 9)


## Spliting the data

Spliting the data in train and test (out-of-time) samples.

Our target is the total purchaseamount for the last 90 days for each customer.

In [5]:
df["date_dt"] = pd.to_datetime(df.date.astype(str), infer_datetime_format=True)

# Threshold for the 90 days observed revenue
test_threshold = df.date_dt.max() - pd.Timedelta(90, unit="days")
train_threshold= test_threshold - pd.Timedelta(90, unit="days")
print(train_threshold, test_threshold)

2012-10-02 00:00:00 2012-12-31 00:00:00


### Test set files

In [11]:
mask = df["date_dt"] >= test_threshold
test_target = df[mask].groupby("id")["purchaseamount"].sum()
test_target = test_target.to_frame().reset_index()
test_target.columns = ["id", "purchaseamount_90d"]
print(test_target.shape)
test_target.head(3)

(307332, 2)


,id,purchaseamount_90d
0,12262064,328.959991
1,12277270,1876.000000
2,12332190,864.099976


In [12]:
test_transactions = df[df["date_dt"] < test_threshold]
print(test_transactions.shape)
test_transactions.head(2)

(206661184, 10)


,id,chain,dept,category,company,brand,date,purchasequantity,purchaseamount,date_dt
0,12262064,95,4,411,102070929,18785,20120302,2,2.50,2012-03-02
1,12262064,95,36,3630,107800070,5208,20120302,1,1.99,2012-03-02


In [17]:
wr.s3.to_parquet(
    df=test_transactions.drop("date_dt", axis=1),
    path="s3://capstone-transaction-data/preprocessed_data/test_transactions.parquet"
)

wr.s3.to_parquet(
    df=test_target,
    path="s3://capstone-transaction-data/preprocessed_data/test_target.parquet"
)

del test_transactions, test_target
gc.collect()

140023

### Train set files

In [18]:
mask = (df["date_dt"] < test_threshold) & (df["date_dt"] >= train_threshold)
train_target = df[mask].groupby("id")["purchaseamount"].sum()
train_target = train_target.to_frame().reset_index()
train_target.columns = ["id", "purchaseamount_90d"]
print(train_target.shape)
train_target.head(3)

(303779, 2)


,id,purchaseamount_90d
0,12262064,838.989990
1,12277270,1505.689941
2,12332190,1084.780029


In [19]:
train_transactions = df[df["date_dt"] < train_threshold]
print(train_transactions.shape)
train_transactions.head(2)

(140440668, 10)


,id,chain,dept,category,company,brand,date,purchasequantity,purchaseamount,date_dt
0,12262064,95,4,411,102070929,18785,20120302,2,2.50,2012-03-02
1,12262064,95,36,3630,107800070,5208,20120302,1,1.99,2012-03-02


In [20]:
wr.s3.to_parquet(
    df=train_transactions.drop("date_dt", axis=1),
    path="s3://capstone-transaction-data/preprocessed_data/train_transactions.parquet"
)

wr.s3.to_parquet(
    df=train_target,
    path="s3://capstone-transaction-data/preprocessed_data/train_target.parquet"
)

{'paths': ['s3://capstone-transaction-data/preprocessed_data/train_target.parquet'],
 'partitions_values': {}}

In [21]:
print(train_transactions.date.min())
print(train_transactions.date.max())

20120302
20121001
